# TF-IDF + Random Forest


### Import data

In [3]:
#load dependencies
import json
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

## Read Data

In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data = []
with open('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
f.close()

/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json
/kaggle/input/dm-2024-isa-5810-lab-2-homework/sampleSubmission.csv
/kaggle/input/dm-2024-isa-5810-lab-2-homework/data_identification.csv
/kaggle/input/dm-2024-isa-5810-lab-2-homework/emotion.csv


NameError: name 'json' is not defined

In [ ]:
# Extract columns like 
df = pd.DataFrame(data)
_source = df['_source'].apply(lambda x: x['tweet'])
df = pd.DataFrame({
    'tweet_id': _source.apply(lambda x: x['tweet_id']),
    'hashtags': _source.apply(lambda x: x['hashtags']),
    'text': _source.apply(lambda x: x['text']),
})

In [ ]:
# Read the rest CSV
emotion = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/emotion.csv')
data_identification = pd.read_csv('/kaggle/input/dm-2024-isa-5810-lab-2-homework/data_identification.csv')

## Data Preprocessing

In [ ]:

# Merge datasets to include identifiers and emotion labels where applicable
df = pd.merge(df, data_identification, on='tweet_id', how='left')

# Split the dataset into training and testing based on the 'identification' column
train_data = df.query("identification == 'train'")
test_data = df.query("identification == 'test'")

# Integrate emotion labels with the training dataset
train_data = pd.merge(train_data, emotion, on='tweet_id', how='left')

# Remove duplicate texts from the training set for cleaner data
train_data = train_data.drop_duplicates(subset='text', keep=False)

# Randomly sample 20% of the training dataset for model training
train_sample = train_data.sample(frac=0.2, random_state=42)

# Extract text (features) and emotion (target) from the sampled dataset
X_train_raw = train_sample['text']
y_train_raw = train_sample['emotion']

## Feature Engineering

In [ ]:
# Split training data into training and validation sets with stratified sampling
from sklearn.model_selection import train_test_split
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_train_raw, y_train_raw, test_size=0.2, random_state=42, stratify=y_train_raw
)

# Convert text data to numerical features using TF-IDF vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=3000)
X_train = tfidf_vectorizer.fit_transform(X_train_raw).toarray()
X_test = tfidf_vectorizer.transform(X_test_raw).toarray()

## Label Encoding

In [ ]:
# Map emotion categories to numerical labels for model training
from sklearn.preprocessing import LabelEncoder
LabelEncode = LabelEncoder()
y_train = LabelEncode.fit_transform(y_train)
y_test = LabelEncode.transform(y_test)

## Training Model (Random_Forest)


In [ ]:
# Initialize and train a Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state=42)
RF.fit(X_train, y_train)

## Model Prediction 

In [ ]:
# Predict on validation set and calculate accuracy
from sklearn.metrics import accuracy_score
y_pred = RF.predict(X_test)
val_accuracy = accuracy_score(y_test, y_pred)
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Prepare test data by transforming its text features with the trained TF-IDF vectorizer
X_test_raw = test_data['text']
X_test = tfidf_vectorizer.transform(X_test_raw).toarray()

# Predict emotions for the test data
y_test_pred = RF.predict(X_test)
y_test_labels = LabelEncode.inverse_transform(y_test_pred)


# Save the Dataframe in specific format
result_df = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_test_labels
})

# Save predictions
result_df.to_csv('/kaggle/working/submission.csv', index=False)

In [7]:
submission.head()

,id,emotion
2,0x28b412,anticipation
4,0x2de201,anticipation
9,0x218443,joy
30,0x2939d5,joy
33,0x26289a,joy
